In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rainflow

In [2]:
# lftab = freqeuncy table / ldtab = input time history / dr = damping ratio / dtime = integration time step(default = 1/(20*fmax))
# it = input type (0=displacement / 1=acceleration)

def th2fds(lftab,ldtab,dr,dtime,it,K,m,C):
    def newmark_parameter(dt):
        r = 0.005; alpha = (1/4)*((1+r)**2); delta = 0.5 + r;
        a = np.zeros(8);
        a[0] = 1/(alpha*dt*dt); a[1] = delta/(alpha*dt); a[2] = 1/(alpha*dt); a[3] = 1/(2*alpha)-1;
        a[4] = delta/alpha - 1; a[5] = (dt/2)*((delta/alpha)-2); a[6] = dt*(1-delta); a[7] = delta*dt;
        return a;
    def DERIV(X,Y):
        N = len(Y); YY = np.zeros(N);
        YY[0] = (Y[1]-Y[0])/(X[1]-X[0]); Y[N-1] = (Y[N-1]-Y[N-2])/(X[N-1]-X[N-2]);
        for n in range(N-2):
            YY[n+1] = (((Y[n+2]-Y[n+1])/(X[n+2]-X[n+1]))*(X[n+1]-X[n]) + ((Y[n+1]-Y[n])/(X[n+1]-X[n]))*(X[n+2]-X[n+1]))/(X[n+2]-X[n]);
        return YY;
    def INTEG(X,Y,C1):
        N = len(Y); YY = np.zeros(N);
        YY[0] = C1;
        for n in range(1,N):
            YY[n] = YY[n-1] + (1/2)*(Y[n]+Y[n-1])*(X[n]-X[n-1]);
        return YY;      
        
    freq = lftab; th = ldtab; zeta = dr; dt = dtime;
    a = newmark_parameter(dt);
    fds = np.zeros(len(freq));
    u,v,ac = np.zeros(len(th)),np.zeros(len(th)),np.zeros(len(th));
    ru,rv,rac = np.zeros(len(th)),np.zeros(len(th)),np.zeros(len(th));
    if it == 0:  # input type = displacement
        N = len(th);
        time = np.arange(N)*dt;
        vth = DERIV(time,th);       
        for fl in range(len(freq)):
            w = 2*np.pi*freq[fl]; m = 1; c = 2*zeta*w; k = w**2;
            F = 2*zeta*w*vth + w**2*th;
            Fe = a[0]*m + a[1]*c + k;
            
            for n in range(len(th-1)):
                u[n+1] = (F[n+1] + m*(a[0]*u[n]+a[2]*v[n]+a[3]*ac[n])+c*(a[1]*u[n]+a[4]*v[n]+a[5]*ac[n]))/Fe;
                v[n+1] = a[1]*(u[n+1]-u[n]) - a[4]*v[n] - a[5]*ac[n];
                ac[n+1] = a[0]*(u[n+1]-u[n]) -a[2]*v[n] - a[3]*ac[n];
            ru = u - th; rv = v - vth;
            st = ru*K;      # Stress Response
            rcc = rainflow.count_cycles(st); # rainflow counting
            rang, cycle = np.zeros(len(rcc)),np.zeros(len(rcc));
            for i in range(len(rcc)):
                rang[i] = rcc[i][0];
                cycle[i] = rcc[i][1];
            fds[fl] = sum((rang**m)*cycle)/C;
    if it == 1: # input type = acceleration
        N = len(th);
        time = np.arange(N)*dt;
        vth = INTEG(time,th,0); dth = INTEG(time,vth,0);
        F = -th;
        for fl in range(len(freq)):
            w = 2*np.pi*freq[fl]; m = 1; c = 2*zeta*w; k = w**2;            
            Fe = a[0]*m + a[1]*c + k;          
            for n in range(len(th)-1):
                ru[n+1] = (F[n+1] + m*(a[0]*ru[n]+a[2]*rv[n]+a[3]*rac[n])+c*(a[1]*ru[n]+a[4]*rv[n]+a[5]*rac[n]))/Fe;
                rv[n+1] = a[1]*(ru[n+1]-ru[n]) - a[4]*rv[n] - a[5]*rac[n];
                rac[n+1] = a[0]*(ru[n+1]-ru[n]) -a[2]*rv[n] - a[3]*rac[n];
            st = ru*K;      # Stress Response
            rcc = rainflow.count_cycles(st); # rainflow counting
            rang, cycle = np.zeros(len(rcc)),np.zeros(len(rcc));
            for i in range(len(rcc)):
                rang[i] = rcc[i][0];
                cycle[i] = rcc[i][1];
            fds[fl] = sum((rang**m)*cycle)/C;
    return freq,fds